In [1]:
import os

In [2]:
origen_libs = '/home/scopatz/origen22/libs'
tape9 = 'pwru50.lib'
lib = os.path.join(origen_libs, tape9)

In [10]:
from pyne.origen22 import parse_tape9
from pyne import nucname

In [4]:
t9 = parse_tape9(lib)

In [5]:
nlb = tuple(sorted(t9.keys()))

In [6]:
nlb

(219, 220, 221)

In [7]:
set(t9[220].keys())

{'_cards',
 '_subtype',
 '_type',
 'fiss_yields_present',
 'sigma_2n',
 'sigma_2n_x',
 'sigma_3n',
 'sigma_f',
 'sigma_gamma',
 'sigma_gamma_x',
 'title'}

In [8]:
t9[219]['sigma_gamma'][30060], t9[221]['sigma_gamma'][30060]

(0.0021870000000000001, 0.0021870000000000001)

In [11]:
XS_RXS = ['gamma', 'z_2n', 'z_3n', 'alpha', 'fission', 'proton', 'gamma_1', 'z_2n_1']
xs_2_o = {'gamma': 'gamma', 'z_2n': '2n', 'z_3n': '3n', 'alpha': 'alpha', 'fission': 'f', 'proton': 'p', 
          'gamma_1': 'gamma_x', 'z_2n_1': '2n_x'}
o_2_xs = {v: k for k, v in xs_2_o.items()}
sigma_maps = {}
for n in nlb:
    for rx in t9[n]:
        if not rx.startswith('sigma_'):
            continue
        for nuc in t9[n][rx]:
            if nuc not in sigma_maps:
                sigma_maps[nuc] = {}
            sigma_maps[nuc]['sigma_' + o_2_xs[rx[6:]]] = t9[n][rx][nuc]

In [13]:
# add absorption xs
for nuc in sigma_maps:
    sigma_maps[nuc]['sigma_a'] = sum(sigma_maps[nuc].values())

In [15]:
nucname.name(nucname.zzaaam_to_id(10010))

'H1'

In [17]:
sigma = {}
for nuc in sigma_maps:
    name = nucname.name(nucname.zzaaam_to_id(int(nuc)))
    for srx in sigma_maps[nuc]:
        sigma[srx + '_' + name] = sigma_maps[nuc][srx]

In [18]:
sigma

{'sigma_a_Se80': 0.10193000000000001,
 'sigma_gamma_Kr87': 46.859999999999999,
 'sigma_proton_Zr94': 3.1080000000000001e-06,
 'sigma_proton_Xe144': 0.0,
 'sigma_fission_Cm247': 24.41,
 'sigma_gamma_1_Eu158': 0.0,
 'sigma_z_2n_1_Rh114': 0.0,
 'sigma_z_2n_Cu65': 0.00022010000000000001,
 'sigma_gamma_1_Ce154': 0.0,
 'sigma_fission_Np239': 0.41649999999999998,
 'sigma_proton_Sb128M': 0.0,
 'sigma_a_Tc101': 0.0,
 'sigma_a_Y96': 0.0,
 'sigma_z_2n_1_Dy163': 0.0,
 'sigma_z_2n_1_Th233': 0.0,
 'sigma_proton_S36': 0.0,
 'sigma_gamma_1_Rh114': 0.0,
 'sigma_proton_Sn121M': 0.0,
 'sigma_gamma_Rb90': 0.0,
 'sigma_a_Xe135': 207200.0,
 'sigma_fission_Cm245': 147.0,
 'sigma_alpha_Na23': 0.00016579999999999999,
 'sigma_proton_Ru102': 0.0,
 'sigma_gamma_1_In133': 0.0,
 'sigma_z_2n_1_Ba138': 0.0,
 'sigma_z_2n_H2': 0.0012080000000000001,
 'sigma_alpha_Cd129': 0.0,
 'sigma_z_2n_Y92': 0.0,
 'sigma_z_2n_Pd108': 0.0,
 'sigma_a_O18': 0.00049671999999999997,
 'sigma_gamma_Ag119': 0.0,
 'sigma_a_Ca43': 0.555864000

In [19]:
import json
with open('sigma.json', 'w') as f:
    json.dump(sigma, f, indent=' ', )